In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline
import torch
from transformers import TextGenerationPipeline
from tqdm.notebook import tqdm
import os
from dataclasses import dataclass, field
from langchain.chains.base import Chain
from datasets import Dataset
from trl.trainer import ConstantLengthDataset
from peft import LoraConfig
from transformers import DataCollatorForLanguageModeling
from trl.trainer import SFTTrainer, SFTConfig
from transformers import Qwen2ForCausalLM, Qwen2TokenizerFast

In [2]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [3]:
def format_options(options):
    return '\n'.join(
        [
            f'{chr(ord("A") + i)}: {option}'
            for i, option in enumerate(options)
        ]
    )

In [4]:
@dataclass
class QuestionItem:
    question: str
    options: list[str]
    reasoning: str | None = field(default=None)
    answer: str | None = field(default=None)

@dataclass
class Entry:
    problem: str = field(default="")
    questions: list[QuestionItem] = field(default_factory=list)

In [5]:
def parse_file(file_path: str) -> list[Entry]:
    with open(file_path, "r") as f:
        lines = f.readlines()
    entries = []
    import json
    for line in lines:
        entry = json.loads(line)
        questions = []
        for question in entry["questions"]:
            questions.append(QuestionItem(**question))
        entries.append(Entry(problem=entry["problem"], questions=questions))
    return entries

In [6]:
from concurrent.futures import ThreadPoolExecutor

In [7]:
def generate_reasoning_inplace(entries: list[Entry]) -> None:
    template = """你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。每个问题都保证能通过一系列基于形式逻辑的推理（包括同一律，矛盾律，排中律的使用等）得到确定的唯一答案。我会向你提供答案，而你要给出逐步的解析来教会我如何得到答案。我是一个小学生，所以每一步的推理不要太复杂。
{problem}

### 问题
{question}
{options}

### 答案
{answer}

### 分析过程"""
    import time
    import qianfan
    client = qianfan.ChatCompletion(model="Qianfan-Chinese-Llama-2-13B")
    progress_q = tqdm(total=sum([len(entry.questions) for entry in entries]))
    import re
    answer_regex = re.compile(r"答案.*?([A-Z])")
    def process_question(entry, question):
        if question.reasoning is not None:
            matches = answer_regex.findall(question.reasoning)
            answer = matches[-1] if len(matches) > 0 else None
            if answer == None:
                print(f"答案未找到: {question.reasoning}")
                # with open("./a.txt", "a") as f:
                #     f.write("1\n")
                #     f.write(question.reasoning)
                # progress_q.update(1)
                # return
            if answer != question.answer:
                print(f"答案不匹配: {answer} != {question.answer}")
            else:
                progress_q.update(1)
                return
        max_retries = 3
        for attempt in range(max_retries):
            try:
                reasoning_prompt = template.format(**{
                    "problem": entry.problem,
                    "question": question.question,
                    "options": format_options(question.options),
                    "answer": question.answer
                })
                from qianfan import QfResponse
                resp = client.do([{
                    "role": "user",
                    "content": reasoning_prompt
                }])
                question.reasoning = resp.body["result"]
                # 答案是 [A-Z]
                import re
                matches = answer_regex.findall(question.reasoning)
                answer = matches[-1] if len(matches) > 0 else None 
                if answer is None:
                    print(f"答案未找到: {question.reasoning}")
                    continue
                if answer != question.answer:
                    print(f"答案不匹配: {answer} != {question.answer}")
                    # Retry
                    continue
                with open("reasoning.txt", "a") as f:
                    f.write(f"{reasoning_prompt}\n\n{question.reasoning}\n\n")
                break  # Exit the loop if successful
            except Exception as e:
                if attempt < max_retries - 1:
                    time.sleep(1)  # Optional: wait a bit before retrying
                else:
                    print(f"Failed to process question.")
                    print(str(e))
        progress_q.update(1)

    with ThreadPoolExecutor(max_workers=1) as executor:
        futures = []
        for entry in tqdm(entries):
            for question in entry.questions:
                futures.append(executor.submit(process_question, entry, question))
        
        for future in futures:
            future.result()
    progress_q.close()

In [8]:
once = False

In [9]:
def create_train_dataset(entries: list[Entry]) -> Dataset:
    dataset = []
    for entry in entries:
        for question in entry.questions:
            if question.reasoning is None or question.answer is None:
                continue
            global once
            if not once:
                print({
                    "problem": entry.problem,
                    "question": question.question,
                    "options": format_options(question.options),
                    "reasoning": question.reasoning,
                    "answer": question.answer
                })
                once = True
            dataset.append({
                "problem": entry.problem,
                "question": question.question,
                "options": format_options(question.options),
                "reasoning": question.reasoning,
                "answer": question.answer
            })
    return Dataset.from_list(dataset)

In [10]:
# entries = parse_file("./round1_train_data.jsonl")
import pickle
entries = pickle.load(open("entries.pkl", "rb"))

In [11]:
entries[1]

Entry(problem='在一场山地自行车比赛中，四位选手分别取得了第一、第二、第三和第四名。不同的颜色代表不同的排名。下面是一些关于这场比赛和选手排名的信息：\n\n1. Alan 名列第一。\n2. 第二名的选手穿红色服装。\n3. John 没有穿黄色服装。\n4. 第四名的选手穿蓝色服装。\n5. Steve 的排名和他的服装色是相同的名次。\n6. Kev 的名次排在 Steve 前面。\n7. 第二名的选手穿的不是青色。\n8. 黄色服穿的选手的成绩排在绿色服穿的选手前面。\n9. 确保每四个参数中的所有元素都不相同和符合排名顺序。\n\n根据上述信息， 回答以下选择题：', questions=[QuestionItem(question='选择题 1:\n根据比赛结果，排在第二名之后的可能是哪些名次？', options=['第一名 第三名', '第三名 第四名'], reasoning='好的，让我们一步一步来分析这个问题。\n\n首先，我们知道 Alan 名列第一（信息 1），所以第一名的位置已经确定了。\n\n接下来，我们知道第二名的选手穿红色服装（信息 2）。\n\n然后，我们知道 John 没有穿黄色服装（信息 3），这意味着 John 不是第二名，因为第二名已经给了红色服装。\n\n现在，我们知道第四名的选手穿蓝色服装（信息 4）。\n\n接着，Steve 的排名和他的服装色是相同的名次（信息 5）。这意味着 Steve 的名次要么是红色（第二名），要么是蓝色（第四名），因为这是我们目前知道的唯一两种颜色。\n\n由于 Kev 的名次排在 Steve 前面（信息 6），那么 Kev 只能是第一名或第二名。但是，Alan 已经名列第一（信息 1），所以 Kev 只能是第二名。这意味着 Steve 不能是第二名，因为 Kev 已经占据了第二名。\n\n因此，Steve 只能是第四名（信息 5 和 6），这意味着他穿蓝色服装。\n\n现在我们知道第二名（红色服装）是 Kev，第四名（蓝色服装）是 Steve。\n\n由于黄色服穿的选手的成绩排在绿色服穿的选手前面（信息 8），而我们已经确定了蓝色和红色服装的选手的位置，所以黄色服装的选手只能是第三名，绿色服装的选手只能是第四名。\n\n所以，排在第二名之后的可能是第三名和第四名。\n\n因此

In [12]:
generate_reasoning_inplace(entries)

  0%|          | 0/687 [00:00<?, ?it/s]

  0%|          | 0/281 [00:00<?, ?it/s]

In [13]:
import re
answer_regex = re.compile(r"答案.*?([A-Z])")
entries_to_drop = []
for entry in entries:
    for question in entry.questions:
        matches = answer_regex.findall(question.reasoning)
        answer = matches[-1] if len(matches) > 0 else None
        if answer != question.answer:
            print(f"答案不匹配: {answer} != {question.answer}")
            entries_to_drop.append(entry)

In [14]:
entries = [entry for entry in entries if entry not in entries_to_drop]

In [15]:
len(entries)

281

In [16]:
ds = create_train_dataset(entries)

{'problem': '有一个英文到法文的词汇表，包含以下对应词汇：\n\n1. the -> le\n2. cat -> chat\n3. jumps -> sauts\n4. over -> sur\n5. moon -> lune\n6. cow -> vache\n7. plays -> jouer\n8. fiddle -> violon\n9. egg -> bougre\n10. falls -> des chutes\n11. off -> de\n12. wall -> mur\n\n根据这个词汇表，翻译以下英文句子成法文：', 'question': '选择题 1：\n英文句子 "the cat jumps over the moon" 翻译成法文是：', 'options': 'A: le chat saute sur la lune\nB: le chat sauts sur le lune\nC: le sauts chat sur le lune\nD: le chat sauts sur le lune', 'reasoning': ' 首先，我们根据英文到法文的词汇表来翻译句子中的每个单词：\n\n1. "the" -> "le"，所以 "the cat" 变成 "le chat"。\n2. "cat" -> "chat"，所以 "cat" 保持不变，依然是 "chat"。\n3. "jumps" -> "sauts"，所以 "jumps over" 变成 "sauts sur"。\n4. "over" -> "sur"，所以 "over the moon" 变成 "sur la lune"。\n\n综上所述，根据词汇表翻译，英文句子 "the cat jumps over the moon" 变成法文是 "le chat sauts sur la lune"。\n\n选项A："le chat saute sur la lune" 错误，因为 "jumps" 对应 "sauts" 而不是 "sautte"。\n选项B："le chat sauts sur le lune" 错误，因为 "over the moon" 应该用 "sur la lune" 而不是 "sur le lune"。\n选项C："le sauts chat su

In [17]:
ds.save_to_disk("./train_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/687 [00:00<?, ? examples/s]

In [18]:
ds[0]

{'problem': '有一个英文到法文的词汇表，包含以下对应词汇：\n\n1. the -> le\n2. cat -> chat\n3. jumps -> sauts\n4. over -> sur\n5. moon -> lune\n6. cow -> vache\n7. plays -> jouer\n8. fiddle -> violon\n9. egg -> bougre\n10. falls -> des chutes\n11. off -> de\n12. wall -> mur\n\n根据这个词汇表，翻译以下英文句子成法文：',
 'question': '选择题 1：\n英文句子 "the cat jumps over the moon" 翻译成法文是：',
 'options': 'A: le chat saute sur la lune\nB: le chat sauts sur le lune\nC: le sauts chat sur le lune\nD: le chat sauts sur le lune',
 'reasoning': ' 首先，我们根据英文到法文的词汇表来翻译句子中的每个单词：\n\n1. "the" -> "le"，所以 "the cat" 变成 "le chat"。\n2. "cat" -> "chat"，所以 "cat" 保持不变，依然是 "chat"。\n3. "jumps" -> "sauts"，所以 "jumps over" 变成 "sauts sur"。\n4. "over" -> "sur"，所以 "over the moon" 变成 "sur la lune"。\n\n综上所述，根据词汇表翻译，英文句子 "the cat jumps over the moon" 变成法文是 "le chat sauts sur la lune"。\n\n选项A："le chat saute sur la lune" 错误，因为 "jumps" 对应 "sauts" 而不是 "sautte"。\n选项B："le chat sauts sur le lune" 错误，因为 "over the moon" 应该用 "sur la lune" 而不是 "sur le lune"。\n选项C："le sauts chat

In [19]:
import pickle
pickle.dump(entries, open("entries.pkl", "wb"))

In [20]:
len(ds)

687